# Modelling

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

import requests
import json
from pandas import json_normalize

import geopandas as gpd
from shapely.geometry import Point

import folium
from folium import plugins
from folium.plugins import HeatMap, MarkerCluster

import h3
from shapely.geometry import Polygon, box, Point  # Import Point from shapely.geometry

# RNN Modeling

## Components

### Sequential Model
- Easy to use and understand, allows to stack layers linearly. This model is appropriate for basic RNNs

### SimpleRNN Layer
- Simplest forms of recurrent layers and effective for simple sequence prediction
- 50 units to prevent model from being overly complex
- Activation Function (ReLU), commonly used activation function that helps model learn non-linear patterns

### Dense Layer
- Single output to predict the next value in time series

### Optimizer and Loss
- Adaptive Moment Estimation (Adam) optimizer for better performance and faster convergence
- Mean Squared Error (MSE) loss is standard for regression and penalizes large errors more than larger ones

In [2]:
# Read the CSV file into a DataFrame
weekday_model = pd.read_csv('assets/processed_data/weekday_model.csv')

# Replace the column name 'Unnamed: 0' with 'h3_index'
weekday_model.columns = weekday_model.columns.str.replace('Unnamed: 0', 'h3_index')

# Set the index of the DataFrame to the second column
weekday_model.set_index(weekday_model.columns[0], inplace=True)

weekday_model

,1.0,3.0,7.0,9.0,10.0,11.0,12.0,14.0,15.0,22.0,...,227.0,228.0,229.0,230.0,231.0,234.0,235.0,242.0,243.0,246.0
h3_index,,,,,,,,,,,,,,,,,,,,,
05:00,0.000000,0.000000,41.783394,11.236842,139.108974,0.000000,0.000000,0.000000,16.873016,33.237589,...,22.184729,34.310030,12.617021,109.661738,0.000000,36.077966,33.716418,187.909091,0.000000,20.940767
06:00,365.400000,23.500000,523.509506,351.523810,560.313830,367.600000,233.937500,1164.000000,182.544379,164.885387,...,169.484700,222.220877,92.960699,522.188433,0.375000,185.143345,202.564706,995.879121,159.923077,176.416382
07:00,40.938776,20.600000,1003.231939,278.000000,1146.800633,77.227273,101.600000,262.666667,246.440644,223.730382,...,192.854727,447.597173,189.516746,1162.903226,0.142857,339.136719,294.020833,1856.820225,95.833333,237.392857
08:00,11.080357,12.894737,1285.225962,271.686047,933.769481,28.830769,9.922330,175.000000,379.043062,259.165306,...,275.838736,496.104129,193.342222,977.573333,0.043478,280.773381,360.847059,1241.363636,41.941176,263.730375
09:00,6.201835,14.421053,563.418251,89.500000,269.614035,9.085106,5.725806,50.750000,203.561776,83.084123,...,145.543829,325.310870,159.487654,547.469136,0.031250,156.789668,133.782875,612.234375,19.380952,103.253247
10:00,9.375000,25.363636,245.382653,43.971660,197.793684,27.717949,6.993103,69.750000,100.325452,62.763789,...,67.542301,205.198758,97.425000,377.759259,0.052632,95.532468,77.083507,534.045455,21.603774,64.756364
11:00,16.250000,31.785714,270.469697,90.751634,188.504310,73.000000,16.024390,82.333333,87.304079,84.579276,...,79.362923,129.641168,82.424242,275.022869,0.138889,92.130000,101.380457,346.477477,39.714286,57.471910
12:00,52.454545,154.666667,352.293413,195.463158,269.398810,180.909091,55.937500,164.500000,101.457931,122.020772,...,124.631873,144.333811,90.783465,255.957968,0.064516,92.242038,188.175258,348.136752,17.384615,88.877083
13:00,30.088235,93.142857,410.179878,166.195402,270.285714,75.166667,37.058824,212.000000,91.927389,106.374821,...,133.339531,153.217327,140.066667,262.719611,0.000000,127.854103,188.027668,274.853333,37.437500,98.080831


In [3]:
# Read the CSV file into a DataFrame
weekend_model = pd.read_csv('assets/processed_data/weekend_model.csv')

# Replace the column name 'Unnamed: 0' with 'h3_index'
weekend_model.columns = weekend_model.columns.str.replace('Unnamed: 0', 'h3_index')

# Set the index of the DataFrame to the second column
weekend_model.set_index(weekend_model.columns[0], inplace=True)

weekend_model.head()

,1.0,3.0,7.0,9.0,10.0,11.0,12.0,14.0,15.0,22.0,...,227.0,228.0,229.0,230.0,231.0,234.0,235.0,242.0,243.0,246.0
h3_index,,,,,,,,,,,,,,,,,,,,,
05:00,0.000000,0.0,18.391667,5.600000,56.286765,0.000000,0.000000,0.0,8.610526,18.024390,...,10.611801,22.793578,7.064516,51.910000,0.0,16.938356,26.576923,71.949153,0.000000,7.568000
06:00,87.875000,17.0,237.974684,52.900000,239.087591,794.000000,306.750000,367.0,90.500000,61.777778,...,98.937008,96.579167,41.333333,189.830688,0.4,73.110092,203.785714,332.463415,176.400000,79.275510
07:00,85.538462,46.0,389.282353,513.625000,389.040323,141.166667,257.875000,195.5,146.959016,95.145455,...,157.264706,181.813953,94.246154,402.786667,0.0,118.873786,169.454545,643.093750,518.000000,139.141176
08:00,27.580645,145.0,536.926829,248.750000,561.444444,65.250000,49.000000,114.0,242.705882,147.482517,...,205.925676,257.366834,142.486486,447.304598,0.2,171.500000,359.115385,922.259259,100.833333,179.684783
09:00,39.920000,38.0,647.818182,133.758621,598.736842,62.000000,46.552632,359.0,267.514493,126.857143,...,181.513453,237.724891,153.772152,506.174699,0.0,137.716667,280.312500,854.250000,106.727273,169.884615


### Weekday

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.layers import Input

# Define window size and forecast horizon
window_size = 3
forecast_horizon = 1

# Create an empty DataFrame to store predictions
predictions_df = pd.DataFrame(columns=weekday_model.columns)

# Function to build the RNN model
def build_rnn_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(SimpleRNN(50, activation='relu'))  # Remove input_shape argument here
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# Iterate through the data using a rolling window approach
for i in range(len(weekday_model) - window_size):
    # Print progress
    print(f"Processing window {i+1}/{len(weekday_model) - window_size}...")
    
    # Extract data within the current window
    window_data = weekday_model.iloc[i:i+window_size]
    
    # Generate predictions for each feature
    predictions = {}
    for feature in window_data.columns:
        # Prepare the data for RNN
        X = window_data[feature].values.reshape((1, window_size, 1))
        y = weekday_model[feature].iloc[i+window_size:i+window_size+forecast_horizon].values.reshape((1, 1))
        
        # Build and train the RNN model
        model = build_rnn_model(X.shape[1:])
        model.fit(X, y, epochs=50, verbose=0)
        
        # Generate prediction
        forecast = model.predict(X)[0][0]
        predictions[feature] = forecast
    
    # Append predictions to the DataFrame with timestamp as index
    timestamp = weekday_model.index[i + window_size]
    predictions_df = predictions_df.append(pd.DataFrame(predictions, index=[timestamp]))

# Reset the index of the predictions DataFrame
predictions_df.reset_index(inplace=True)
predictions_df.rename(columns={'index': 'timestamp'}, inplace=True)

Processing window 1/16...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1

C:\Users\IINDR\AppData\Local\Temp\ipykernel_9772\4087974633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append(pd.DataFrame(predictions, index=[timestamp]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

C:\Users\IINDR\AppData\Local\Temp\ipykernel_9772\4087974633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append(pd.DataFrame(predictions, index=[timestamp]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

...

C:\Users\IINDR\AppData\Local\Temp\ipykernel_9772\4087974633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append(pd.DataFrame(predictions, index=[timestamp]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

C:\Users\IINDR\AppData\Local\Temp\ipykernel_9772\4087974633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append(pd.DataFrame(predictions, index=[timestamp]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

C:\Users\IINDR\AppData\Local\Temp\ipykernel_9772\4087974633.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append(pd.DataFrame(predictions, index=[timestamp]))


In [5]:
predictions_df

,timestamp,1.0,3.0,7.0,9.0,10.0,11.0,12.0,14.0,15.0,...,227.0,228.0,229.0,230.0,231.0,234.0,235.0,242.0,243.0,246.0
0,08:00,12.326084,14.167109,973.636353,288.250458,599.704895,32.065525,10.523396,190.728775,370.571564,...,251.629486,319.108154,81.850464,706.993408,0.045017,182.264832,385.961060,1324.769043,42.640202,208.760910
1,09:00,5.190256,15.209727,534.890686,93.837929,279.906311,9.838885,3.893584,54.825474,203.664429,...,158.523972,357.980988,152.694748,553.059509,0.031617,156.173050,135.602097,615.830017,17.232441,96.408783
2,10:00,9.391261,21.488907,245.655136,45.258739,193.520798,29.946880,6.635819,65.591011,101.483452,...,67.158783,194.649826,93.309921,368.639648,0.052707,93.556808,78.049141,518.188354,21.762228,62.256531
3,11:00,14.293860,28.172117,272.009460,92.293800,192.815277,51.771187,17.430252,85.225441,85.535164,...,75.633339,130.607056,78.701408,278.748810,0.130814,87.420433,96.570099,337.767548,40.476059,59.176678
4,12:00,37.916279,46.748489,359.801788,171.473526,292.967163,29.242655,26.318710,173.624451,103.892563,...,134.535889,149.354416,93.812134,260.780518,0.064290,93.651695,189.221375,327.571686,17.342712,87.829102
5,13:00,31.855322,96.955528,442.274536,129.265427,279.580078,73.294556,32.120541,168.328247,99.728264,...,140.577911,163.262329,57.795620,264.517670,0.000451,124.669106,164.288620,258.856720,35.972908,74.530098
6,14:00,32.786774,120.857925,426.553680,135.891281,257.178986,57.763287,32.711792,118.549805,124.389503,...,164.185562,138.379349,120.955070,255.448654,0.494682,111.837105,140.846909,304.752960,51.288757,107.115257
7,15:00,36.069042,90.126823,371.388123,131.168701,268.998566,136.998917,42.764675,96.096092,135.257736,...,184.491028,149.524597,132.717560,221.551361,0.435119,92.508995,92.689507,258.126129,40.760567,105.759689
8,16:00,86.242645,327.171387,449.169464,202.735397,274.491608,159.831482,57.618973,63.241711,154.575943,...,289.191864,161.260178,142.929977,215.552582,0.150402,86.907120,210.765549,254.924194,72.108543,124.465172
9,17:00,36.048134,321.141388,623.614014,243.774490,187.466995,301.400513,33.802364,109.020607,305.936554,...,484.131683,302.587585,178.963547,341.957367,0.554619,130.111694,569.821716,438.559784,82.778702,217.627594


In [6]:
rnn_weekday = predictions_df.copy()

In [7]:
rnn_weekday.index = rnn_weekday['timestamp']
rnn_weekday = rnn_weekday.drop(columns = ['timestamp'])

In [8]:
weekday_actual = weekday_model[3:]
weekday_actual.shape

(16, 113)

In [9]:
# Calculate RMSE for each feature
rmse_values = []
for column in rnn_weekday.columns:
    squared_diff = (weekday_actual[column] - rnn_weekday[column]) ** 2
    mse = squared_diff.mean()
    rmse_values.append(np.sqrt(mse))

# Average out the RMSE values
average_rmse = np.mean(rmse_values)

print("Average RMSE across all features:", average_rmse)

Average RMSE across all features: 62.668496312544015


In [10]:
rnn_weekday.to_csv('assets/processed_data/rnn_weekday_50.csv')

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.layers import Input

# Define window size and forecast horizon
window_size = 3
forecast_horizon = 1

# Create an empty DataFrame to store predictions
weekend_predictions_df = pd.DataFrame(columns=weekend_model.columns)

# Function to build the RNN model
def build_rnn_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(SimpleRNN(50, activation='relu'))  # Remove input_shape argument here
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# Iterate through the data using a rolling window approach
for i in range(len(weekend_model) - window_size):
    # Print progress
    print(f"Processing window {i+1}/{len(weekend_model) - window_size}...")
    
    # Extract data within the current window
    window_data = weekend_model.iloc[i:i+window_size]
    
    # Generate predictions for each feature
    predictions = {}
    for feature in window_data.columns:
        # Prepare the data for RNN
        X = window_data[feature].values.reshape((1, window_size, 1))
        y = weekend_model[feature].iloc[i+window_size:i+window_size+forecast_horizon].values.reshape((1, 1))
        
        # Build and train the RNN model
        model = build_rnn_model(X.shape[1:])
        model.fit(X, y, epochs=50, verbose=0)
        
        # Generate prediction
        forecast = model.predict(X)[0][0]
        predictions[feature] = forecast
    
    # Append predictions to the DataFrame with timestamp as index
    timestamp = weekday_model.index[i + window_size]
    weekend_predictions_df = weekend_predictions_df.append(pd.DataFrame(predictions, index=[timestamp]))

# Reset the index of the predictions DataFrame
weekend_predictions_df.reset_index(inplace=True)
weekend_predictions_df.rename(columns={'index': 'timestamp'}, inplace=True)

Processing window 1/16...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━

C:\Users\IINDR\AppData\Local\Temp\ipykernel_19020\2025157862.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weekend_predictions_df = weekend_predictions_df.append(pd.DataFrame(predictions, index=[timestamp]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

C:\Users\IINDR\AppData\Local\Temp\ipykernel_19020\2025157862.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weekend_predictions_df = weekend_predictions_df.append(pd.DataFrame(predictions, index=[timestamp]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

...

C:\Users\IINDR\AppData\Local\Temp\ipykernel_19020\2025157862.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weekend_predictions_df = weekend_predictions_df.append(pd.DataFrame(predictions, index=[timestamp]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

C:\Users\IINDR\AppData\Local\Temp\ipykernel_19020\2025157862.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weekend_predictions_df = weekend_predictions_df.append(pd.DataFrame(predictions, index=[timestamp]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

C:\Users\IINDR\AppData\Local\Temp\ipykernel_19020\2025157862.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weekend_predictions_df = weekend_predictions_df.append(pd.DataFrame(predictions, index=[timestamp]))


In [5]:
weekend_predictions_df

,timestamp,1.0,3.0,7.0,9.0,10.0,11.0,12.0,14.0,15.0,...,227.0,228.0,229.0,230.0,231.0,234.0,235.0,242.0,243.0,246.0
0,08:00,29.256376,27.427320,364.674652,249.907364,371.977173,62.415783,48.355957,111.432549,129.199295,...,148.540588,174.984711,85.219925,363.317596,0.206210,117.117401,257.960480,462.539948,95.686157,148.919312
1,09:00,37.549988,41.774292,631.245178,130.142715,504.844666,63.893753,46.919720,354.676483,257.661407,...,191.097839,260.322510,132.580383,413.835846,0.002876,149.987991,291.862122,862.564819,107.259415,184.695816
2,10:00,86.044357,95.369644,615.946533,189.090454,569.643555,66.769913,61.584805,122.221710,258.723816,...,198.757889,245.808929,81.247528,413.215210,0.532505,166.669418,340.157776,496.728455,116.737709,185.257248
3,11:00,59.792233,61.763103,764.700623,231.830154,497.691833,206.333878,84.453468,319.055695,252.443863,...,235.230865,223.896957,112.622131,529.152283,0.056153,131.792282,521.561523,600.123718,92.027168,273.070862
4,12:00,139.822647,60.772522,612.889954,193.641785,526.936401,161.854889,137.355194,225.597672,190.677185,...,212.341782,243.659821,117.067818,556.389771,0.115950,142.757874,535.977417,530.720093,57.534443,267.268219
5,13:00,297.418854,51.104130,710.362976,153.812927,400.963470,100.645279,111.836914,258.437012,126.906525,...,170.218750,241.021317,98.340096,482.965790,0.385741,123.893608,289.741272,409.547394,35.568516,155.313217
6,14:00,266.740051,30.978437,534.795471,157.651321,367.835876,229.904617,135.069626,218.613037,126.555588,...,198.310226,193.013535,96.089333,419.962311,0.898603,109.907173,377.420227,307.717957,164.856522,130.836914
7,15:00,123.560989,58.317276,512.437317,262.672333,350.880127,229.092621,88.328369,194.850998,186.496536,...,278.110077,177.054611,93.538399,428.575684,0.541036,123.937881,312.266876,374.842407,105.214134,110.957191
8,16:00,196.159760,24.046183,691.072815,225.049362,343.024750,286.499268,116.920502,73.904587,248.743454,...,347.576508,213.123383,73.309105,430.346313,0.840460,139.570496,316.208984,484.256042,210.860809,140.869400
9,17:00,194.875931,88.828957,959.192566,590.506958,549.645508,810.074707,288.417358,326.237885,293.732727,...,522.134521,304.796906,192.000931,517.357544,0.123912,198.802597,594.794495,663.827637,199.123657,220.779022


In [6]:
rnn_weekend = weekend_predictions_df.copy()

In [7]:
rnn_weekend.index = rnn_weekend['timestamp']
rnn_weekend = rnn_weekend.drop(columns = ['timestamp'])

In [8]:
rnn_weekend.shape

(16, 113)

In [9]:
weekend_actual = weekend_model[3:]
weekend_actual.shape

(16, 113)

In [12]:
# Calculate RMSE for each feature
weekend_rmse_values = []
for column in rnn_weekend.columns:
    squared_diff = (weekend_actual[column] - rnn_weekend[column]) ** 2
    mse = squared_diff.mean()
    weekend_rmse_values.append(np.sqrt(mse))

# Average out the RMSE values
weekend_average_rmse = np.mean(weekend_rmse_values)

print("Average RMSE across all features:", weekend_average_rmse)

Average RMSE across all features: 48.978595211917806


In [13]:
rnn_weekend.to_csv('assets/processed_data/rnn_weekend_50.csv')

### Plotting

In [18]:
average_rmse

46.65760268074464

In [20]:
rnn_weekday = pd.read_csv('assets/processed_data/rnn_weekday_50.csv')

# Replace the column name 'Unnamed: 0' with 'h3_index'
rnn_weekday.columns = rnn_weekday.columns.str.replace('Unnamed: 0', 'h3_index')

# Set the index of the DataFrame to the second column
rnn_weekday.set_index(rnn_weekday.columns[0], inplace=True)

rnn_weekday.head()

,1.0,2.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,18.0,...,218.0,219.0,225.0,226.0,229.0,231.0,235.0,236.0,237.0,243.0
timestamp,,,,,,,,,,,,,,,,,,,,,
08:00,513.831360,200.154630,538.79530,11.394221,814.93060,206.63277,134.498100,134.11348,22.611506,421.11273,...,13.972423,38.587257,356.63892,162.857060,6.815423,40.551070,34.223446,11.178165,37.071630,7.420184
09:00,317.131300,64.886050,250.32753,7.155099,601.42480,83.09894,157.325090,319.27286,28.459509,280.39630,...,7.328227,15.950479,249.76427,66.443436,2.538421,31.094748,17.827322,9.870170,42.371630,5.569912
10:00,161.219530,73.929310,174.89641,7.985016,366.83417,88.06438,94.366684,144.68600,17.177330,190.59150,...,8.885279,21.052452,166.90547,52.015990,3.846019,30.892935,10.097050,17.722307,109.414600,9.870482
11:00,117.280914,108.777145,142.15941,11.618899,271.37402,189.00485,98.801150,106.77162,38.232143,153.70921,...,17.708940,40.477253,127.01492,59.896810,5.775725,30.870914,28.434164,39.652866,126.797226,7.443244
12:00,142.184460,100.532320,131.35535,10.649557,247.91562,282.86960,96.773790,129.08482,39.387590,151.88297,...,29.172756,85.284130,148.85315,55.559650,1.717373,40.678470,16.936764,23.124765,86.834600,9.695772


In [27]:
weekday_model.iloc[3:].head()

,1.0,3.0,7.0,9.0,10.0,11.0,12.0,14.0,15.0,22.0,...,227.0,228.0,229.0,230.0,231.0,234.0,235.0,242.0,243.0,246.0
h3_index,,,,,,,,,,,,,,,,,,,,,
08:00,11.080357,12.894737,1285.225962,271.686047,933.769481,28.830769,9.922330,175.000000,379.043062,259.165306,...,275.838736,496.104129,193.342222,977.573333,0.043478,280.773381,360.847059,1241.363636,41.941176,263.730375
09:00,6.201835,14.421053,563.418251,89.500000,269.614035,9.085106,5.725806,50.750000,203.561776,83.084123,...,145.543829,325.310870,159.487654,547.469136,0.031250,156.789668,133.782875,612.234375,19.380952,103.253247
10:00,9.375000,25.363636,245.382653,43.971660,197.793684,27.717949,6.993103,69.750000,100.325452,62.763789,...,67.542301,205.198758,97.425000,377.759259,0.052632,95.532468,77.083507,534.045455,21.603774,64.756364
11:00,16.250000,31.785714,270.469697,90.751634,188.504310,73.000000,16.024390,82.333333,87.304079,84.579276,...,79.362923,129.641168,82.424242,275.022869,0.138889,92.130000,101.380457,346.477477,39.714286,57.471910
12:00,52.454545,154.666667,352.293413,195.463158,269.398810,180.909091,55.937500,164.500000,101.457931,122.020772,...,124.631873,144.333811,90.783465,255.957968,0.064516,92.242038,188.175258,348.136752,17.384615,88.877083


In [ ]:
import matplotlib.pyplot as plt

# Iterate through each column in weekend_model
for column in weekday_model.columns:
    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(weekday_model.index, weekday_model[column], label='Actual')
    plt.plot(rnn_weekday.index, rnn_weekday[column], label='Predicted')
    plt.title(f"Actual vs Predicted for {column}")
    plt.xlabel("Timestamp")
    plt.ylabel("Value")
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()